# Исследование надежности заемщиков.

При выдаче кредита банки сталкиваются с рядом рисков. Скоринговая модель банка поможет снизить уровень рисков.
Для построения скоринговой модели необходимо понять, влияет ли семейное положение, количество детей, уровень дохода и цель займа на факт погашения кредита в срок.

**Цель исследования** - определить платежеспособность клиентов по следующим показателям.


### Шаг 1. Обзор данных

In [ ]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv') #чтение файла с сохранением в переменную data
data.head(10) #просмотр первых 10 строк таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info() #просмотр общей информации таблицы data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Таблица `data` содержит двенадцать столбцов. Типы данных используемых в таблицах: `float64`, `int64` и `object`.
Согласно описанию данных:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В столбцах `days_employed` и `total_income`количество значений отличается от общего количества. Это значит, что в этих столбцах имеются пропущенные значения.  
Так же в столбце `days_employed` отрицательные значения сразу бросаются в глаза, но т.к. столбец указывает на *общий трудовой стаж в днях*. Это говорит о том, что в данных есть аномалии, которые необходимо исправить до заполнения пропусков.


### Шаг 2.1 Заполнение пропусков

In [ ]:
data.isna().sum() #подсчет пропусков в таблице

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
round(data.isna().sum() * 100 / len(data),1) #подсчет пропусков от всех значений датафрейма

children             0.0
days_employed       10.1
dob_years            0.0
education            0.0
education_id         0.0
family_status        0.0
family_status_id     0.0
gender               0.0
income_type          0.0
debt                 0.0
total_income        10.1
purpose              0.0
dtype: float64

Количество пропусков в столбцах `days_employed` и `total_income` одинаковы и составляют более 10% от всех значений датафрейма. Такая цифра достаточно весома и отметает вариант с удалением строк с пропущенными значениями.

Расмотрим пропущенные данные подробнее.

In [ ]:
data[data['days_employed'].isna()].head(10) #найдём все строки с пропусками в столбце days_employed и просмотрим первые 10

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [ ]:
data[data['total_income'].isna()].head(10) #найдём все строки с пропусками в столбце days_employed и просмотрим первые 10

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


После вывода первых 10 строк с пропушенными значениями по столбцам `days_employed` и `total_income`, оказалось, что эти строки индентичны. 
<br>Можно допустить, что значения `NaN`в столбцах `days_employed` и `total_income` находятся в одних и тех же строках. Отсюда можно предположить, что природа появления `NaN` в таблице имеет технический характер.

<br>*Общий трудовой стаж в днях* и *ежемесячный доход* это количественные переменные. Для заполнения пропусков в количественных переменных можно использовать, например,**среднее арифметическое** или **медиану**.



In [ ]:
round(data['total_income'].describe(),3)

count      19351.000
mean      167422.302
std       102971.566
min        20667.264
25%       103053.153
50%       145017.938
75%       203435.068
max      2265604.029
Name: total_income, dtype: float64

Мы видим, что разброс между минимальными и максимальными значениями столбца `total_income` довольно большие. В таком случае, медиана даст более реалистичную цифру.   
Медиану считают лучше, чем среднее значение, если если разница между максимальным и минимальным значениями большие.Среднее значение, было бы подходящим, если данные были распределены симметрично.   
Произведем замену данных в столбце `total_income` медиальным значением с учетом типа занятости - `income_type`.



In [ ]:
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type'])['total_income'].transform('median'))
#замена пропущенных значений медиальными по столбцу 'total_income'
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Вызвав информационное окно, мы видим, что пропуски в столбце `total_income `заполнены.

### Шаг 2.2 Проверка данных на аномалии и исправления.

Ранее мы нашли аномалию в столбце `days_employed`.      
<br>Выведем первые 10 строк с аномалией:

In [ ]:
data.loc[data['days_employed'] < 0,].head(10) 
# вывод первых 10 строк таблицы с отрицательным значением в столбце 'days_employed'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи
10,2,-4171.483647,36,высшее,0,женат / замужем,0,M,компаньон,0,113943.491460,покупка недвижимости


In [ ]:
round(data.loc[data['days_employed'] < 0, 'days_employed'].count() * 100 / len(data),1) #находим процент отрицательных значений

73.9

Общее количество отрицательных значений составляет 73,9% от общего количества строк в таблице. Такой большой процент наводит на мысль, что возникла техническая проблема при загрузке данных и '-' к значениям добавлен случайно.
Отбросим минуса в столбце `days_employed`:


In [ ]:
data.loc[data['days_employed'] != 'NaN', 'days_employed'] = abs(data['days_employed'])#используем модуль для избавления от отрицательных значений стажа
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Для удобства восприятия информации о стаже преведем её в годы и выведем таблицу на экран:

In [ ]:
data['days_employed'] = round(data['days_employed']/365,3) #перевод дней трудового стажа в годы
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.117,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.027,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.407,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.301,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,932.236,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.419,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18.986,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.997,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


После перевода стажа в годы стало легче воспринимать информацию. В таблице теперь видно, что наблюдаются так же  аномалии и некоторые значения стажа явно завышены. Скорее всего сработал *человеческий фактор* и заполняющий перепутал столбцы или единицы измерения.<br> Приведем в соответсвие завышенные значения с учетом возраста. Предварительно рассмотрим более подробно возрастные категории имеющиеся в таблице. 

In [ ]:
round(data['dob_years'].describe(),3) #просмотр основных статистических данных столбца 'dob_years'

count    21525.000
mean        43.293
std         12.575
min          0.000
25%         33.000
50%         42.000
75%         53.000
max         75.000
Name: dob_years, dtype: float64

Мы видим, минимальный возраст равен 0, это значит что часть клиентов не правильно заполнили данное поле. Рассмотрим эти строки более подробно.

In [ ]:
data.loc[data['dob_years'] == 0].head(10) #вывод первых 10 строк со значением 0 в столбце 'dob_years'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,949.429,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,7.299,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,5.131,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,1090.018,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,3.173,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,2.561,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,1016.108,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,13.817,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,142594.396847,жилье
1898,0,1014.095,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


Заменим нулевые значения возраста на средние значения возраста по типу занятости.

In [ ]:
income_unic = data.loc[data['dob_years'],'income_type'].unique() 
#уникальные значения типа занятости, к которых присутствует возраст равный 0.
income_unic

array(['сотрудник', 'компаньон', 'пенсионер', 'госслужащий'], dtype=object)

In [ ]:
data.loc[((data['income_type'] == 'пенсионер')&(data['dob_years'] == 0)), 'dob_years']  = data.loc[data['income_type'] == 'пенсионер', 'dob_years'].median()
data.loc[((data['income_type'] == 'сотрудник')&(data['dob_years'] == 0)), 'dob_years']  = data.loc[data['income_type'] == 'сотрудник', 'dob_years'].median()
data.loc[((data['income_type'] == 'компаньон')&(data['dob_years'] == 0)), 'dob_years']  = data.loc[data['income_type'] == 'компаньон', 'dob_years'].median()
data.loc[((data['income_type'] == 'госслужащий')&(data['dob_years'] == 0)), 'dob_years']  = data.loc[data['income_type'] == 'госслужащий', 'dob_years'].median()

In [ ]:
round(data['dob_years'].describe(),3) #просмотр основных статистических данных столбца 'dob_years' после замены нулевых значений возраста

count    21525.000
mean        43.496
std         12.232
min         19.000
25%         34.000
50%         43.000
75%         53.000
max         75.000
Name: dob_years, dtype: float64

Минимальное значение стажа изменилось и теперь равно 19 годам. <br>Заменим теперь аномально большие значения стажа в соответствии с возрастом клиентов. Примем максимально возможный стаж как разницу между возрастом клиента и 18 годами (возраст совершенолетия). 

In [ ]:
data.loc[(data['days_employed'] > data['dob_years'] - 18), 'days_employed']  = data['dob_years'] - 18 #замена аномально больших згачений стажа
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.117,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.027,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.000,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.301,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,35.000,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.419,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,17.000,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.997,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


 <br> Заменим теперь значения `NaN`в столбце`days_employed` с учетом типа занятости.

In [ ]:
data['days_employed'] = data['days_employed'].fillna(data.groupby(['income_type'])['days_employed'].transform('median'))
#замена Nan на медиану с учетом типа занятости 'income_type' 
data.head(10)#вывод таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.117,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.027,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.000,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.301,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,35.000,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.419,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,17.000,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.997,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info()#вывод окна информации

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


После вызова окна информации видно, что `Nan` значений больше нет.
<br> Проверим столбец `children` на аномалии. Для начала выведем основную статистическую информацию:

In [ ]:
round(data['children'].describe(),3) #просмотр основных статистических данных столбца children

count    21525.000
mean         0.539
std          1.382
min         -1.000
25%          0.000
50%          0.000
75%          1.000
max         20.000
Name: children, dtype: float64

Максимальное количество детей равно 20, что вполне допустимо, однако минимальное значение равно -1, что не возможно. Посмотрим, где встречается отрицательное значение  в количестве детей.

In [ ]:
data[data['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,2.412,21.0,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,2.344,44.0,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,9.070,56.0,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,7.436,59.0,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,5.922,39.0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,3.398,40.0,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,1.647,37.0,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,4.194,53.0,среднее,1,женат / замужем,0,M,компаньон,0,172357.950966,покупка жилой недвижимости
21404,20,1.356,52.0,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


In [ ]:
display(data[data['children'] == -1].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,12.103,46.0,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,2.471,50.0,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,8.697,57.0,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,36.000,54.0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,42.000,57.0,Среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,на покупку своего автомобиля
1363,-1,3.275,55.0,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,4.004,38.0,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,6.958,42.0,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,8.000,26.0,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,2.469,41.0,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


Строк, в которых встречается отрицательное количество детей несколько. Скорее всего такие значения возникли при ошибке в подгрузке файла или не правильном заполнении форм.<br> Воспользуемся модулем, что бы заменить отрицательные значения на положительные.

In [ ]:
data.loc[data['children'] == -1, 'children'] = abs(data['children'])
data['children'].sort_values().unique() #сортировка и вывод уникальных значений столбца children для проверки

array([ 0,  1,  2,  3,  4,  5, 20])

Отрицательное количество детей из столбца заменены.

### Шаг 2.3. Изменение типов данных.

In [ ]:
data['total_income'] = data['total_income'].astype('int') 
#замена вещественного типа данных на целочисленный в столбце 'total_income' 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Проверим наличие дубликатов в таблице.
<br> Посмотрим сначала количество явных дубликатов с помощью `duplicated()` и `sum()`

In [ ]:
data.duplicated().sum() # подсчёт явных дубликатов

54

Мы нашли 54 дубликата. Посмотрим первые 10 строк с дубликатами

In [ ]:
duplicated_df = data[data.duplicated()] #создаем датафрейм с дубликатами
duplicated_df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,4.305,41.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,4.305,34.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,42.000,60.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,42.000,58.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,42.000,57.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,42.000,58.0,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
9238,2,4.305,34.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9528,0,42.000,66.0,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9627,0,42.000,56.0,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
10462,0,42.000,62.0,среднее,1,женат / замужем,0,F,пенсионер,0,118514,покупка коммерческой недвижимости


Подобного типа дубликаты могли возникнуть из-за того, что клиент дважды заполнил одну и ту же заявку или же при загрузке фрагмента в таблицу, его могли загрузить несколько раз. <br> Удалим янвные дубликаты и сформируем новые индексы.

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21471 non-null  int64  
 1   days_employed     21471 non-null  float64
 2   dob_years         21471 non-null  float64
 3   education         21471 non-null  object 
 4   education_id      21471 non-null  int64  
 5   family_status     21471 non-null  object 
 6   family_status_id  21471 non-null  int64  
 7   gender            21471 non-null  object 
 8   income_type       21471 non-null  object 
 9   debt              21471 non-null  int64  
 10  total_income      21471 non-null  int64  
 11  purpose           21471 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Количество строк в таблице сократилось. Делаем вывод, что дубликаты удалены. 
<br>
<br>Проверим датафрейм на неявные дубликаты. Для этого извлечем нужный столбец для проверки, отсортируем и вернем его уникальные значения c помощью функций `sort_values()` и `unique()`. <br> Начнем со столюца `education`

In [ ]:
data['education'].sort_values().unique() #сортировка для поиска не явных дубликатов столбца 'education'

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

Мы видим, что в столбце `education` есть записи с разными регистрами. Чтобы учесть такие дубликаты, переведем все записи к нижнему регистру вызовом метода `lower()`

In [ ]:
data['education'] = data['education'].str.lower() #приведение к нижнему регистру

Проверим столбец `education` еще раз, что бы убедиться, что дубликаты с разными регистрами заменились на один вид записи. <br> Проверим данные на явные дубликаты еще раз, что бы проверить не появились ли новые после изменения регистра в столбце `education`.

In [ ]:
data.duplicated().sum() # подсчёт явных дубликатов

17

Мы видим, что у нас появилось 17 новых явных дубликатов. Удалим их:

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
data['education'].sort_values().unique() 
#сортировка для поиска не явных дубликатов столбца 'education' после удаления дубликатов

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

Мы видим, что в столбце `education` дубли удалены. Проверим столбцы: `family_status`, `gender`, `purpose`

In [ ]:
data['family_status'].sort_values().unique() #сортировка для поиска не явных дубликатов

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

В столбце `family_status` дублей нет.

In [ ]:
data['gender'].sort_values().unique() #сортировка для поиска неявных дубликатов

array(['F', 'M', 'XNA'], dtype=object)

В столбце `gender` помимо стандартных значений 'F' и 'M' присутствует значение 'XNA'. Просмотрим строки с этим значением.

In [ ]:
display(data[data['gender'] == 'XNA']) #вывод строк со значением  'XNA' на экран

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10684,0,6.0,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Найдена только одна строка со значением 'XNA' в столбце `gender`. Т.к.строка только одна и на определение зависимости погашения кредита в срок от семейного положения и количества детей она не влияет, принимаем решение не изменять значение 'XNA'.
<br> Проверим дубли в столбце `purpose`:

In [ ]:
data['purpose'].sort_values().unique() #сортировка для поиска не явных дубликатов

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

После изучения уникальных значений в столбце `purpose` принято решение, в дальнейшем, разделить цели получения кредита по категориям.
<br>
<br>Для поиска явных дубликатов мы использовали метод `duplicated()`, котрый по умолчанию считает дубликатами строки, которые полностью повторяют те, что уже встречались в таблице. С помощью `sum()`, мы посчитали количество строк-дубликатов. Для просмотра этих строк мы создали датафрейм `duplicated_df`.
<br>Для удаления явных дубликатов мы использовали метод `drop_duplicates()`. Совместно с методом `drop_duplicates()` был изпользован метод `reset_index(drop=True)`. Благодаря этим методам мы не только удалили дубликаты, но исбросили индексы строк с удалением старых.
<br>
<br>В случае неявных дубликатов мы использовали приведение в столбце `education` значний к единому регистру с помощью функции `lower()`. Благодаря этому количество уникальных значений в столбце сократилось с 15 до 5.
<br><br>**Причины возникновения дебликатов в данных**
<br>Дубликаты могут возникнуть по нескольким причинам. Например, когда нет четко сформулированных правил заполнения анкет. В данном датасете примером этому может служить различное написание образования:'ВЫСШЕЕ', 'Высшее'и 'высшее'. Возможно, четкая инструкция по заполнению анкеты или выпадающий список с уже имеющимися значениями могли бы решить эту проблему.
<br>Возникновение явных дубликатов может произойти, например, из-за повторной загрузки данных в базу. Так же подобные дубликаты могут возникнуть, если клиент несколько раз отправлял одинаковую информацию о себе.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Для облегчения работы с данными в таблице создадим отдельные "словари" для столбцов `education` и `family_status`

In [ ]:
education_log = data[['education_id','education']] #создаем таблицу-"словарь" для столбца 'education'
education_log = education_log.drop_duplicates().reset_index(drop=True) #удаляем дубли в строках таблицы
education_log.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [ ]:
family_log = data[['family_status_id','family_status']] #создаем таблицу-"словарь" для столбца 'family_status'
family_log = family_log.drop_duplicates().reset_index(drop=True)#удаляем дубли в строках таблицы
family_log.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


В результате у нас появились две таблицы `education_log` и `family_log` со значением и ее идентификационным номером. Теперь можем удалить столбцы `education` и `family_status` из основной таблицы `data`.

In [ ]:
#data = data.drop(columns=['education','family_status'], errors='ignore') #удаление столбцов с параметром errors= ‘ignore’ для предотвращения возникновение ошибокtry:
try:#удаление столбцов c применением try-except
    data = data.drop(columns=['education','family_status'])
except:
    display('Есть ошибки, но мы все равно удалим ненужные столбцы')


In [ ]:
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.117,42.0,0,0,F,сотрудник,0,253875,покупка жилья
1,1,11.027,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15.000,33.0,1,0,M,сотрудник,0,145885,покупка жилья
3,3,11.301,32.0,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,35.000,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,2.537,27.0,0,1,M,компаньон,0,255763,покупка жилья
6,0,7.888,43.0,0,0,F,компаньон,0,240525,операции с жильем
7,0,0.419,50.0,1,0,M,сотрудник,0,135823,образование
8,2,17.000,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,5.997,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи


Благодаря выполненым действиям упрощается визуальная работа с таблицей, для обращения к значениям из столбцов `ducation` и `family_status` достаточно узнать их 'id', а не писать полностью наименование.

### Шаг 2.6. Категоризация дохода.

Разделим кредитополучателей на сегменты по уровню дохода и создадим столбец `total_income_category`, в котором будет информация у какому сегменту клиент находится. <br>
Категории:
 - 0–30000 — `'E'`;
 - 30001–50000 — `'D'`;
 - 50001–200000 — `'C'`;
 - 200001–1000000 — `'B'`;
 - 1000001 и выше — `'A'`.

In [ ]:

def total_income_category(income): #функция, присваивающая категорию
    if 0 <= income <= 30000:
        return 'E'
    if  30001 <= income <=50000:
        return 'D'
    if  50001 <= income <=200000:
        return 'C'
    if  200001 <= income <=1000000:
        return 'B'
    
    return 'A' 
data['total_income_category'] = data['total_income'].apply(total_income_category)
#создание нового столбца с указанием категории
data['total_income_category'].value_counts() #вывод статистики по категориям уровян дохода

C    16015
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Самой большой категорие является категория `'C'` с заработной платой от 50001 до 200000. <br> Наименьшей является категория `'E'` c заработной платой от 0 до 30000. 

### Шаг 2.7. Категоризация целей кредита.

Необходимо произвести сегментацию по столбцу `purpose` и привязать его значения к определенным категориям. <br> Расмотрим еще раз уникальные значения столбца `purpose`.

In [ ]:
data['purpose'].value_counts()# простомр того, сколько раз встречается каждая цель получения кредита

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Все значения в столбце можно разделить на следующие категории:
 - операции с автомобилем
 - операции с недвижимостью
 - проведение свадьбы
 - получение образования

In [ ]:
def purpose_category(purposes): #функция, присваивающая категорию
     
    if 'свадьб' in purposes:
        return 'проведение свадьбы'
    if 'автом' in purposes:
        return 'операции с автомобилем' 
    if 'образов' in purposes:
        return 'получение образования'
    return 'операции с недвижимостью'   
        
data['purpose_category'] = data['purpose'].apply(purpose_category)#создание нового столбца с указанием категории

In [ ]:
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,23.117,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,11.027,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,15.000,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,11.301,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,35.000,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,2.537,27.0,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,7.888,43.0,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,0.419,50.0,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,17.000,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,5.997,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [ ]:
data['purpose_category'].value_counts() #вывод статистики по категориям уровян дохода

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

Наиболее часто кредит брали на операции с недвижимостью, на операции с автомобилем и получение образрвания число взятик кредита почти одинаково. Самой маленькой категорией янвляется взятие кредина на проведение свадьбы.

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Создадим сводную таблицу `data_pivot` в которой выведем процент случаев с проблемой в погашении кредита от общего количества взятых кретитов в зависимости от количества детей.

In [ ]:
data_pivot = data.pivot_table(index=['children'], columns='debt', values='total_income_category', aggfunc='count')
# создание сводной таблицы
data_pivot['sum'] = data_pivot[1]+data_pivot[0]
data_pivot['%'] = round(data_pivot[1]/(data_pivot[1]+data_pivot[0])*100,1) 
#добавление столбца с процентным соотношением 
data_pivot #вывод таблицы

debt,0,1,sum,%
children,,,,
0,13028.0,1063.0,14091.0,7.5
1,4410.0,445.0,4855.0,9.2
2,1858.0,194.0,2052.0,9.5
3,303.0,27.0,330.0,8.2
4,37.0,4.0,41.0,9.8
5,9.0,NaN,NaN,NaN
20,68.0,8.0,76.0,10.5


##### Вывод 1:

Мы видим что процентный разброс между категориями с количеством детей не большой.<br> Наибольший процент неуплат в срок приходится на клиентов у кого 20 детей.<br> Наименьший процент у клиентов не имеющих детей.<br> <br>Зависимость между количеством детей и возвратом кредита в срок можно проследить в первых 3х строках, с увеличением количества детей процент неуплаты повышется. По 3-6 строкам сделать выводы сложно, т.к. таких кредитов мало в сравнении с общим количеством.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим таблицу `data_subcategory_family` в которой выведем процент случаев с проблемой в погашении кредита от общего количества взятых кретитов в зависимости от семейного статуса.

In [ ]:
data_pivot_2 = data.pivot_table(index=['family_status_id'], columns='debt', values='total_income_category', aggfunc='count')
# создание сводной таблицы
data_pivot_2['sum'] = data_pivot_2[1]+data_pivot_2[0]
data_pivot_2['%'] = round(data_pivot_2[1]/(data_pivot_2[1]+data_pivot_2[0])*100,1)
#добавление столбца с процентным соотношением 
data_subcategory_family = data_pivot_2.merge(family_log, on='family_status_id',how='left')
#присоединение расшифровки семейного статуса
data_subcategory_family #вывод таблицы

,family_status_id,0,1,sum,%,family_status
0,0,11408,931,12339,7.5,женат / замужем
1,1,3763,388,4151,9.3,гражданский брак
2,2,896,63,959,6.6,вдовец / вдова
3,3,1110,85,1195,7.1,в разводе
4,4,2536,274,2810,9.8,Не женат / не замужем


##### Вывод 2:

Процентный разброс неуплаты креди в срок в зависимости от семеного статуса не большой. Наименьший прошент неуплат приходится на категорию *вдовец / вдова* и составляет **6.6%** Наибольший на *Не женат / не замужем* - **9.8%** <br> <br> Исходя из полученной нами таблицы наиболее платежеспособными являются люди с семейным положением: `вдовец / вдова`, `в разводе`,`женат / замужем`.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим сводную таблицу `data_pivot_3` в которой выведем процент случаев с проблемой в погашении кредита от общего количества взятых кретитов в зависимости от дохода клиента.

In [ ]:
data_pivot_3 = data.pivot_table(index=['total_income_category'], columns='debt', values='income_type', aggfunc='count')
# создание сводной таблицы
data_pivot_3['sum'] = data_pivot_3[1]+data_pivot_3[0]
data_pivot_3['%'] = round(data_pivot_3[1]/(data_pivot_3[1]+data_pivot_3[0])*100,1)
#добавление столбца с процентным соотношением 
data_pivot_3

debt,0,1,sum,%
total_income_category,,,,
A,23,2,25,8.0
B,4686,356,5042,7.1
C,14655,1360,16015,8.5
D,329,21,350,6.0
E,20,2,22,9.1


##### Вывод 3:

Наименьший процент с проблемами по выплате кредита приходится на клиентов с доходом категории `D`- *от 30001 до 50000*. Наибольший приходится на категорию `E`- *от 0 до 30000*. <br> Однако, из-за того, что по категориям `A`,`D` и`E` количество кредитов очень мало, поэтому сделать вывод затруднительно.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Создадим сводную таблицу `data_pivot_4` в которой выведем процент случаев с проблемой в погашении кредита от общего количества взятых кретитов в зависимости от цели кредита.

In [ ]:
data_pivot_4 = data.pivot_table(index=['purpose_category'], columns='debt', values='income_type', aggfunc='count')
# создание сводной таблицы
data_pivot_4['sum'] = data_pivot_4[1]+data_pivot_4[0]
data_pivot_4['%'] = round(data_pivot_4[1]/(data_pivot_4[1]+data_pivot_4[0])*100,1)
#добавление столбца с процентным соотношением 
data_pivot_4

debt,0,1,sum,%
purpose_category,,,,
операции с автомобилем,3903,403,4306,9.4
операции с недвижимостью,10029,782,10811,7.2
получение образования,3643,370,4013,9.2
проведение свадьбы,2138,186,2324,8.0


##### Вывод 4:

В таблице `data_pivot_4` наблюдается зависмость между целью кредита и выплатой его в срок. Наиболее надежным является выдача кредита под *операции с недвижимостью*. А *операции с автомобилями* и *полечение образования* являются более рискованными целями для выдачи кредита.

## Общий вывод:

Мы дали ответы на поставленные вопросы:<br><br>
**1) Зависит ли уплата кредита в срок от количества детей?<br><br>**
В предоставленных данных зависимость прослеживается для клиентов имеющих 0-2 детей. Чем больше детей, тем больше риски просрочки уплаты.<br><br>
**2) Зависит ли уплата кредита в срок от семейного положения?**<br><br>
Зависимость в данных прослеживается. Наиболее платежеспособными являются вдовцы и люди состоящие в браке.<br><br>
**3) Зависит ли уплата кредита в срок от уровня дохода?**<br><br>
В предоставленных данных сложно дать ответ на данный вопрос, т.к. распределение данных не равномерно и основной процент выдачи кредитов приходится на категории `C`(*от 50001 до 200000*) и `B`(*от 200001 до 1000000*) 74% и 23% соответственно.<br><br>
**4) Зависит ли уплата кредита в срок от целей на которые он был взят?**
Из 4х категорий имеющихся в данных наиболее надежный заемщик будет брать кредит на операции с недвижимость, а наименее надежный на автомобиль. Зависимость от целей кредита прослеживается. <br><br>
Рассматривая предоставленные данные можно составить *портрет "идеального" заемщика*. <br> "Идеальный" заемщик - Вдовец/вдова имеющий 5 детей, уровень дохода от 30001 до 50000 и собирающийся производить операции с недвижимостью. <br>Однако, если оперировать не только цифрами, но и логикой, то *"идеальный заемщик"* становится - Женатым/замужним человеком, без детей с заработком  от 200001 до 1000000 и так же, собирающийся производить операции с недвижимостью.
<br>Объединяет эти два портрета два аспекта: 
<br>1) Клиент был или сейчас женат/замужем. Скорее всего это делает человека более собранным и обязательным.
<br>2) Клиент собирается производить операции с недвижимостью. Недвижимость является, довольно, стабильным вложением. Что бы получить кредит на недвижимость необходимо предоставить больше документов. Зачастую банки прописывают в договорах, что при просрочке платежей, возможно одностороннее повышение процента. Клиенту это не выгодно и он старается платить в срок.